# Data Transformation

## Table A. Energy Supply & Consumption Balance

In [6]:
import pandas as pd

# Load both sheets
xls = pd.ExcelFile("../data/raw/table_a_supply_consumption.xlsx")

# Optional: Check names if not sure
print(xls.sheet_names)


['Title page', '2021-22', '2022-23']


In [7]:
# Step 1: Define flow type mapping (simplified to start)
flow_map = {
    "Primary indigenous supply": "primary_supply",
    "plus all imports": "imports",
    "less all exports": "exports",
    "less stock changes": "stock_changes",
    "less discrepancies": "statistical_discrepancies",
    "Total primary energy supply": "primary_energy_supply",

    "Coke ovens": "conversion",
    "LNG plants": "conversion",
    "Petroleum refining": "conversion",
    "Gas manufacturing": "conversion",
    "Electricity generation": "conversion",
    "Other conversion": "conversion",
    "Fuel use in conversion": "conversion_losses",

    "Total final energy consumption": "final_consumption",

    "Agriculture": "sector_agriculture",
    "Mining": "sector_mining",
    "Food, beverages, textiles": "sector_industry",
    "Wood, paper And printing": "sector_industry",
    "Chemical": "sector_industry",
    "Iron and steel": "sector_industry",
    "Non-ferrous metals": "sector_industry",
    "Other industry": "sector_industry",
    "Water and waste": "sector_services",
    "Construction": "sector_services",
    "Transport": "sector_transport",
    "Commercial and services": "sector_commercial",
    "Residential": "sector_residential",
    "Lubes, bitumen, solvents": "sector_other"
}

In [17]:
sheets = {"2021-22": "2021-22", "2022-23": "2022-23"}
dfs = []

for sheet_name, year in sheets.items():
    df = xls.parse(sheet_name=sheet_name, skiprows=4)
    df = df.dropna(axis=1, how='all')
    
    # Clean column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^a-z0-9_]', '', regex=True)
    id_col = df.columns[0]
    
    # Drop total row and nans
    df = df[~df[id_col].str.lower().str.contains("total", na=False)]
    df = df.dropna(subset=[id_col])
    
    # Assign flow_type based on matching
    def assign_flow(label):
        for key, val in flow_map.items():
            if key.lower() in str(label).lower():
                return val
        return "unclassified"

    df["flow_type"] = df[id_col].apply(assign_flow)
    df["period"] = year
    
    # Melt to long format
    df_melt = df.melt(id_vars=[id_col, "flow_type", "period"], var_name="fuel", value_name="petajoules")
    dfs.append(df_melt)



In [18]:
# Combine and save
df_final = pd.concat(dfs, ignore_index=True)
df_final = df_final.dropna(subset=["petajoules"])
df_final = df_final.rename(columns={"unnamed_1": "flow_description"})
df_final.head()


,flow_description,flow_type,period,fuel,petajoules
0,Primary indigenous supply,primary_supply,2021-22,black_coal,11215.668
1,plus all imports,imports,2021-22,black_coal,2.636
2,less all exports,exports,2021-22,black_coal,10173.475
3,less stock changes,stock_changes,2021-22,black_coal,-202.904
4,less discrepancies,statistical_discrepancies,2021-22,black_coal,66.587


In [85]:
df_final.to_csv("../data/processed/table_a_energy_supply_consumption.csv", index=False)
print("Data processing complete. Output saved to ../data/processed/energy_supply_consumption.csv")

Data processing complete. Output saved to ../data/processed/energy_supply_consumption.csv


## Table B. Population GDP Consumption

In [21]:
xls = pd.ExcelFile("../data/raw/table_b_population_gdp_consumption.xlsx")
print("Sheets:", xls.sheet_names)

Sheets: ['Title page', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT']


In [75]:
import pandas as pd

def load_table_b_clean(path, sheet_code, sheet_map):
    """
    Cleans one sheet from Table B (Population, GDP, and Energy Consumption).
    Combines title + unit for column names, cleans year, adds state.
    """
    df_all = pd.read_excel(path, sheet_name=sheet_code, header=None)

    # Extract column headers from row 4 and units from row 5
    title_row = df_all.iloc[4]
    unit_row = df_all.iloc[5]

    # Build combined column names
    combined_columns = []
    for idx, (title, unit) in enumerate(zip(title_row, unit_row)):
        title = str(title).strip()
        unit = str(unit).strip()

        if idx == 1:
            combined_columns.append("year")
        elif title and title.lower() != "nan":
            col_name = f"{title} [{unit}]" if unit and unit.lower() != "nan" else title
            combined_columns.append(col_name)
        else:
            combined_columns.append(f"unnamed_{idx}")

    # Load data starting from row 6 onward
    df = df_all.iloc[6:].copy()
    df.columns = combined_columns

    # Drop the first unnamed column if it exists (e.g., "unnamed_0")
    if "unnamed_0" in df.columns:
        df = df.drop(columns=["unnamed_0"])

    # Keep only rows with a valid year
    df = df[df["year"].astype(str).str.match(r"\d{4}-\d{2}", na=False)]

    # Convert "1962-63" to 1962 (int)
    df["year"] = df["year"].astype(str).str.extract(r"^(\d{4})").astype(int)

    # Add state column
    df["state"] = sheet_map.get(sheet_code, sheet_code)

    return df

In [82]:
# Sheet mapping
sheet_map = {
    "AUS": "Australia",
    "NSW": "New South Wales",
    "VIC": "Victoria",
    "QLD": "Queensland",
    "SA": "South Australia",
    "WA": "Western Australia",
    "TAS": "Tasmania",
    "NT": "Northern Territory"
}

# File path
path = "../data/raw/table_b_population_gdp_consumption.xlsx"

# Clean and combine all sheets
all_dfs = []
for sheet_code in sheet_map:
    df_clean = load_table_b_clean(path, sheet_code, sheet_map)
    all_dfs.append(df_clean)

df_b_final = pd.concat(all_dfs, ignore_index=True)


df_b_final.columns = [
    col.lower().strip().replace(" ", "_") if "[" not in col else
    col.split("[")[0].strip().lower().replace(" ", "_") + " [" + col.split("[")[1].lower()
    for col in df_b_final.columns
]


# Save
df_b_final.to_csv("../data/processed/table_b_population_gdp_consumption.csv", index=False)
print("Table B cleaned and saved to: data/processed/table_b_population_gdp_consumption.csv")

Table B cleaned and saved to: data/processed/table_b_population_gdp_consumption.csv


In [83]:
df_b_final.head()

,year,population [number],gdp [$ million],energy_consumption [pj],energy_consumption_per_capita [gj/person],energy_intensity [gj/$ million],energy_productivity [$ million/pj],state
0,1960,10548267,308362,1336.6,126.713,4334.516,230.706,Australia
1,1961,10742291,312353,1365.8,127.142,4372.617,228.696,Australia
2,1962,10950379,331770,1432.2,130.79,4316.846,231.651,Australia
3,1963,11166702,354929,1531.4,137.14,4314.666,231.768,Australia
4,1964,11387665,376155,1625.3,142.725,4320.825,231.437,Australia


## Table C. Consumption by Fuel

In [87]:
def load_table_c_clean(path, sheet_code, sheet_map):
    """
    Cleans one sheet from Table C (Consumption by Fuel).
    Combines title + unit, cleans year, adds state.
    """
    import pandas as pd

    df_all = pd.read_excel(path, sheet_name=sheet_code, header=None)

    # Extract column titles (row 4) and units (row 5)
    title_row = df_all.iloc[4]
    unit_row = df_all.iloc[5]

    # Build clean column names
    combined_columns = []
    for idx, (title, unit) in enumerate(zip(title_row, unit_row)):
        title = str(title).strip()
        unit = str(unit).strip()

        if idx == 1:
            combined_columns.append("year")
        elif title and title.lower() != "nan":
            col_name = f"{title} [{unit}]" if unit and unit.lower() != "nan" else title
            combined_columns.append(col_name.lower().replace(" ", "_"))
        else:
            combined_columns.append(f"unnamed_{idx}")

    # Load data from row 6 onward
    df = df_all.iloc[6:].copy()
    df.columns = combined_columns

    # Drop the unnamed_0 column (filler)
    if "unnamed_0" in df.columns:
        df = df.drop(columns=["unnamed_0"])

    # Filter valid year rows
    df = df[df["year"].astype(str).str.match(r"\d{4}-\d{2}", na=False)]

    # Convert to 4-digit year
    df["year"] = df["year"].astype(str).str.extract(r"^(\d{4})").astype(int)

    # Add state column
    df["state"] = sheet_map.get(sheet_code, sheet_code)

    return df

In [88]:
sheet_map = {
    "AUS": "Australia",
    "NSW": "New South Wales",
    "VIC": "Victoria",
    "QLD": "Queensland",
    "SA": "South Australia",
    "WA": "Western Australia",
    "TAS": "Tasmania",
    "NT": "Northern Territory"
}

path = "../data/raw/table_c_consumption_by_fuel.xlsx"
dfs = []

for sheet_code in sheet_map:
    df_clean = load_table_c_clean(path, sheet_code, sheet_map)
    dfs.append(df_clean)

df_c_final = pd.concat(dfs, ignore_index=True)

# Optional: normalize column names to snake_case + keep units
df_c_final.columns = [
    col.lower().strip().replace(" ", "_") if "[" not in col else
    col.split("[")[0].strip().lower().replace(" ", "_") + " [" + col.split("[")[1].lower()
    for col in df_c_final.columns
]

# Save final cleaned table
df_c_final.to_csv("../data/processed/table_c_consumption_by_fuel.csv", index=False)
print("Table C cleaned and saved.")

Table C cleaned and saved.


In [89]:
df_c_final.head()

,year,coal_ [pj],oil_ [pj],gas_ [pj],renewables_ [pj],statistical_discrepancy_b_ [pj],total_ [pj],state
0,1960,na,510,0,189.1,na,1336.6,Australia
1,1961,na,537.3,0,185.6,na,1365.8,Australia
2,1962,na,581.9,0.1,194.6,na,1432.2,Australia
3,1963,na,656.8,0.1,188.9,na,1531.4,Australia
4,1964,na,719.9,0.1,198.2,na,1625.3,Australia


## Table D. Consumption Detailed Fuel

In [133]:
def clean_table_d_by_state(path: str, sheet_name: str, state_name: str) -> pd.DataFrame:
    import pandas as pd

    # Read entire sheet
    df_all = pd.read_excel(path, sheet_name=sheet_name, header=None)

    # Slice from row 7 down (actual data)
    df = df_all.iloc[7:].copy()

    # Assign clean column names manually
    df.columns = [
        "drop_0", "year_raw",
        "black_coal [pj]", "brown_coal [pj]", "coke [pj]", "coal_by_products [pj]",
        "liquid_gas_biofuels [pj]", "wood_woodwaste [pj]", "bagasse [pj]",
        "refinery_input [pj]", "petroleum_products [pj]", "natural_gas [pj]",
        "town_gas [pj]", "electricity [pj]", "solar_hotwater [pj]", "drop_15",
        "coke_derived [pj]", "coal_by_products_derived [pj]", "petroleum_products_a [pj]",
        "town_gas_derived [pj]", "thermal_electricity [pj]", "drop_21",
        "total_energy_consumption [pj]"
    ]

    # Extract 4-digit year
    df["year"] = df["year_raw"].astype(str).str.extract(r"^(\d{4})")
    df["year"] = df["year"].astype("Int64")

    # Drop unused columns
    df_clean = df.drop(columns=["drop_0", "year_raw", "drop_15", "drop_21"])

    # Melt to long format
    df_long = df_clean.melt(id_vars="year", var_name="fuel", value_name="pj")

    # Assign category
    df_long["category"] = df_long["fuel"].apply(
        lambda x: "consumption_of_fuels" if "derived" not in x and "total" not in x
        else ("production_of_derived_fuels" if "derived" in x else "total")
    )

    # Clean fuel name
    df_long["fuel"] = df_long["fuel"].str.replace("_derived", "", regex=False)

    # Convert pj to float
    df_long["pj"] = pd.to_numeric(df_long["pj"], errors="coerce")
    df_long.rename(columns={"pj": "energy_[pj]"}, inplace=True)

    # Add state
    df_long["state"] = state_name

    return df_long

In [134]:
# Map sheet codes to readable state names
sheet_map = {
    "AUS": "Australia",
    "NSW": "New South Wales",
    "VIC": "Victoria",
    "QLD": "Queensland",
    "SA": "South Australia",
    "WA": "Western Australia",
    "TAS": "Tasmania",
    "NT": "Northern Territory"
}

path = "../data/raw/table_d_consumption_detailed_fuel.xlsx"
all_states_df = []

# Loop through each sheet/state
for sheet_code, state_name in sheet_map.items():
    print(f"Processing: {state_name}")
    df_state = clean_table_d_by_state(path, sheet_code, state_name)
    all_states_df.append(df_state)

# Combine all
df_d_final = pd.concat(all_states_df, ignore_index=True)

# Save final dataset
df_d_final.to_csv("../data/processed/table_d_consumption_detailed_fuel_all_states.csv", index=False)
print("All states processed and saved.")

Processing: Australia
Processing: New South Wales
Processing: Victoria
Processing: Queensland
Processing: South Australia
Processing: Western Australia
Processing: Tasmania
Processing: Northern Territory
All states processed and saved.


In [135]:
df_d_final.head()

,year,fuel,energy_[pj],category,state
0,1960,black_coal [pj],495.3,consumption_of_fuels,Australia
1,1961,black_coal [pj],492.3,consumption_of_fuels,Australia
2,1962,black_coal [pj],494.3,consumption_of_fuels,Australia
3,1963,black_coal [pj],517.0,consumption_of_fuels,Australia
4,1964,black_coal [pj],531.5,consumption_of_fuels,Australia
